# Credit VAR of a Swap

The goal of the project is to compute the Credit VAR (95th percentile) of a swap in which the counterparty might default. The input parameters are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* A set of forward libor rates and their fixing dates
* The static data of the swap (nominal, start date, maturity, underlying libor tenor, etc..). The fixed rate of the swap is such that if there was no counterparty risk, the product would be traded at par. You are requested to compute it.
* A set of survival probabilities for the risky counterparty
* The recovery rate for this swap (i.e. the fraction of the market value that it is expected to recover in case of default).
* The volatility of the swap rate, which is assumed to be constant through time (the swap rate is lognormal with fixed volatility)

The swap starts today and the horizon for the VAR calculation is before any future payments. 

The requested VAR is today’s value at risk, i.e. the 95th percentile of the discounted loss distribution function.

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for credit risk evaluation.

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!


In [2]:
from datetime import date

In [3]:
pricing_date = date(2013, 10, 31)

In [4]:
survival_probabilities = [
    {'date': date(2014, 12, 20), 'ndp': 0.972159727015014},
    {'date': date(2015, 12, 20), 'ndp': 0.942926329174406},
    {'date': date(2016, 12, 20), 'ndp': 0.913448056250137},
    {'date': date(2018, 12, 20), 'ndp': 0.855640452819766},
    {'date': date(2023, 12, 20), 'ndp': 0.732687779675469},
    {'date': date(2033, 12, 20), 'ndp': 0.539046016487758},
]

In [5]:
ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

In [6]:
euribor_6m = [
    {'date': date(2013, 10, 31), 'rate': 0.005},
    {'date': date(2014, 4, 30), 'rate': 0.0058},
    {'date': date(2014, 10, 30), 'rate': 0.0066},
    {'date': date(2015, 4, 30), 'rate': 0.0074},
    {'date': date(2015, 10, 30), 'rate': 0.0082},
    {'date': date(2016, 4, 30), 'rate': 0.009},
    {'date': date(2016, 10, 30), 'rate': 0.0098},
    {'date': date(2017, 4, 30), 'rate': 0.0106},
    {'date': date(2017, 10, 30), 'rate': 0.0114},
    {'date': date(2018, 4, 30), 'rate': 0.0122},
    {'date': date(2018, 10, 30), 'rate': 0.013},
    {'date': date(2019, 4, 30), 'rate': 0.0138},
    {'date': date(2019, 10, 30), 'rate': 0.0146},
    {'date': date(2020, 4, 30), 'rate': 0.0154},
    {'date': date(2020, 10, 30), 'rate': 0.0162},
    {'date': date(2021, 4, 30), 'rate': 0.017},
    {'date': date(2021, 10, 30), 'rate': 0.0178},
    {'date': date(2022, 4, 30), 'rate': 0.0186},
    {'date': date(2022, 10, 30), 'rate': 0.0194},
    {'date': date(2023, 4, 30), 'rate': 0.0202},
    {'date': date(2023, 10, 30), 'rate': 0.021},
    {'date': date(2024, 4, 30), 'rate': 0.0218},
    {'date': date(2024, 10, 30), 'rate': 0.0226},
    {'date': date(2025, 4, 30), 'rate': 0.0234},
    {'date': date(2025, 10, 30), 'rate': 0.0242},
    {'date': date(2026, 4, 30), 'rate': 0.025},
    {'date': date(2026, 10, 30), 'rate': 0.0258},
    {'date': date(2027, 4, 30), 'rate': 0.0266},
    {'date': date(2027, 10, 30), 'rate': 0.0274},
    {'date': date(2028, 4, 30), 'rate': 0.0282},
]

In [7]:
irs_to_price = {
    'nominal': 100e6, 
    'libor_tenor': 6,
    'fixed_leg_frequency': 12,
    'maturity': 120
}

In [32]:
credit_var_date = date(2014, 1, 31)
notional = 100e6
tenor = 6
maturity = 120


In [9]:
recovery = 0.4

In [10]:
volatility = 0.2

In [11]:
mc_runs = 10000

In [12]:
percentile = 0.95

In [14]:
curve_6m_pillar = []
curve_6m_rate = []
for item in euribor_6m:
    curve_6m_pillar.append(item['date'])
    curve_6m_rate.append(item['rate'])

In [16]:
from finmarkets import ForwardRateCurve
libor_curve = ForwardRateCurve(curve_6m_pillar, curve_6m_rate)

In [20]:
from finmarkets import OvernightIndexSwap, generate_swap_dates
pillar_dates = [pricing_date]

swaps=[]

for quote in ois_quotes:
    swap = OvernightIndexSwap(
        100e6,
        
        # payment dates
        generate_swap_dates(
            pricing_date,
            quote['maturity']
        ),
        
        # the fixed rate
        quote['rate']
        
    )
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

n_df_vector = len(pillar_dates)

In [25]:
from finmarkets import DiscountCurve

def objective_function(x):
    curve = DiscountCurve(
        # today date
        pricing_date,
        
        # pillar dates
        pillar_dates,
        
        # pillar discount factors
        x
        
    )
    
    sum_sq = 0.0
    
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [26]:
from scipy.optimize import minimize
x0 = [1.0 for i in range(n_df_vector)]
bounds = [(0.01, 100.0) for i in range(n_df_vector)]

# constrain the first pillar discount factor = 1
bounds[0] = (1.0, 1.0)

result = minimize(objective_function, x0, bounds=bounds)

In [29]:
discount_curve = DiscountCurve(pricing_date, pillar_dates, result.x)

In [30]:
import numpy as np

def expon_icdf(p, lambd=1):
    return -np.log(1-p)/lambd

In [33]:
from finmarkets import InterestRateSwap

irs_random=InterestRateSwap(pricing_date, notional, 0.1, tenor, maturity)
swap_rate=irs_random.swap_rate(discount_curve, libor_curve)
swap_rate

0.02605698895129351

In [36]:
irs_fair = InterestRateSwap(pricing_date, notional, swap_rate, tenor, maturity)

In [38]:
A=irs_fair.annuity(discount_curve)
S=irs_fair.swap_rate(discount_curve, libor_curve)

In [66]:
from finmarkets import CreditCurve

pillar_dates = []
rate_dates=[]

for quote in survival_probabilities:  
        pillar_dates.append(quote['date']) 
for rate in survival_probabilities:
        rate_dates.append(rate['ndp'])

credit_curve = CreditCurve(pillar_dates, rate_dates)
print(pricing_date)
credit_curve.ndp(datetime.date(2013,10,31))

2013-10-31


0.972159727015014

In [43]:
from dateutil.relativedelta import relativedelta

numdays = (pricing_date + relativedelta(months=120) - pricing_date).days


In [47]:
date_list = [pricing_date + datetime.timedelta(days=x) for x in range(0, numdays)] 


In [57]:
epsilon = []

d = date_list[0]
print (d)
while d <= date_list[-1]:
    hazard_rate = credit_curve.hazard(d)
    epsilon.append(hazard_rate/365)
    d += relativedelta(days=1)
print (epsilon[1])

2013-10-31
-0.0


In [53]:
hazard_function = []
for i in range(len(epsilon)):
    hazard_function.append(sum(epsilon[:i]))
    
hazard_function

[0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,


In [54]:
values_hazard_function=[]

for item in hazard_function:
    K = 1 - np.exp(-item)
    values_hazard_function.append(K)
values_hazard_function

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0